In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
import json # library to handle JSON files


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import urllib.request


import time
from bs4 import BeautifulSoup

import re
print("imported")

imported


In [2]:
#getting data
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(page.text, 'html.parser')

#variables for table, table_header, rows and data etc
the_table = soup.find('table')
table_header = the_table.findAll('th')
rows = the_table.findAll('tr')
data = the_table.findAll('td')
 
#removing html tags from headers    
for i, head in enumerate(table_header): table_header[i]=str(table_header[i]).replace("<th>","").replace("</th>","").replace("\n","")

#removing html tags from rows   
for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")

# creating dataframe
df=pd.DataFrame(rows)

# make dataframe, expand rows and drop the old one:
df[table_header] = df[0].str.split("</td>\n<td>", 2, expand = True) 
df.drop(columns=[0],inplace=True)

#rename column name 
df.rename(columns={'Postal Code':'PostalCode'},inplace = True)

#droping the first row
df = df.drop(0)

#filtering out table--removing not assigned rows
df=df[df['Borough'] != 'Not assigned\n']

# remove \n tag from data
df.update(
    df.Borough.loc[
        lambda x: x.str.contains('\n')
    ].str.replace("\n",""))


df.update(
    df.PostalCode.loc[
        lambda x: x.str.contains('\n')
    ].str.replace("\n",""))

# replace "not assigned" Nieghborhood with data from Borogh
df.Neighborhood.replace("Not assigned", df.Borough, inplace=True)

df.head(10)




,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,"Malvern, Rouge"
12,M3B,North York,Don Mills
13,M4B,East York,"Parkview Hill, Woodbine Gardens"
14,M5B,Downtown Toronto,"Garden District, Ryerson"


In [3]:
#add Geo-spatial data
df_geoData= pd.read_csv("http://cocl.us/Geospatial_data")
df_geoData.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_geoData.head
df_geoData.set_index("Postcode")

#rename column as that of geodta to make a join
df.rename(columns={'PostalCode':'Postcode'}, inplace=True)
df.set_index("Postcode")

#merge geospatial data, nieghbourhood data 
toronto_data=pd.merge(df,df_geoData, on='Postcode', how='inner')
toronto_data


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [4]:
!conda install -c conda-forge geopy --yes  
from geopy.geocoders import Nominatim 

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed and imported!


In [5]:
# Toronto latitude and longitude values
latitude = 43.6532
longitude = -79.3832
# create map and display it
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# display the map of San Francisco
toronto_map

In [25]:
# instantiate a feature group for the neighbourhoods in the dataframe
Neighborhoods = folium.map.FeatureGroup()

# loop through the latitude and longitue and add on the map
for lat, lng, in zip(toronto_data.Latitude, toronto_data.Longitude):
    Neighborhoods.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
toronto_map.add_child(incidents)
toronto_map

In [26]:
#get all teh neighborhoods of North York borough
northYork_data=toronto_data[toronto_data['Borough'].str.contains("North York")]
northYork_data.head()


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
7,M3B,North York,Don Mills,43.745906,-79.352188
10,M6B,North York,Glencairn,43.709577,-79.445073


In [8]:
#get latitute and longitude of Victorial Village--a test to see that code is working
neighborhood_latitude = northYork_data.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = northYork_data.loc[1, 'Longitude'] # neighborhood longitude value

# Victoria village neighbourhood
neighborhood_name = northYork_data.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Victoria Village are 43.725882299999995, -79.31557159999998.


In [9]:
#call foursqaure api fucntion to get venues
CLIENT_ID = 'W3IIYS2EGJEW0ZZZMYXXXXXXXXXXXXXXXXXXXX' #  Foursquare ID
CLIENT_SECRET = '20RPHPBQY43G2OCV2YYYYYYYYYYYYYYYY' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius=500
LIMIT =100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ebe61b6ed78b8001b9072cb'},
 'response': {'headerLocation': 'Bermondsey',
  'headerFullLocation': 'Bermondsey, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.7303823045, 'lng': -79.30935618239715},
   'sw': {'lat': 43.72138229549999, 'lng': -79.32178701760282}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c633acb86b6be9a61268e34',
       'name': 'Victoria Village Arena',
       'location': {'lat': 43.72348055545508,
        'lng': -79.31563520925143,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.72348055545508,
          'lng': -79.31563520925143}],
        'distance': 267,
        'cc': 'CA',
        'country': 'Canada',
        'formatte

In [27]:

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\yasin\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Victoria Village Arena,Hockey Arena,43.723481,-79.315635
1,Tim Hortons,Coffee Shop,43.725517,-79.313103
2,Portugril,Portuguese Restaurant,43.725819,-79.312785
3,Pizza Nova,Pizza Place,43.725824,-79.312860


In [28]:
#fucntion to get nearby venue---Source Coresera capstone Lab 
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
northYork_venues = getNearbyVenues(names=northYork_data['Neighborhood'],
                                   latitudes=northYork_data['Latitude'],
                                   longitudes=northYork_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale
Downsview
York Mills West
Willowdale


In [14]:
northYork_venues.head(2)
#northYork_venues.shape


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop


In [15]:
#northYork_venues.groupby('Neighborhood').count()

In [16]:
#Analyze Each Neighborhood--source of the source code: Coursera capstone Lab
northYork_onehot = pd.get_dummies(northYork_venues['Venue Category'], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northYork_onehot['Neighborhood'] = northYork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northYork_onehot.columns[-1]] + list(northYork_onehot.columns[:-1])
northYork_onehot = northYork_onehot[fixed_columns]

northYork_onehot.head(2)

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
northYork_onehot.shape

(241, 99)

In [18]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶
northYork_grouped = northYork_onehot.groupby('Neighborhood').mean().reset_index()
northYork_grouped.head(2)

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10,0.0,...,0.05,0.05,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0
1,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0


In [19]:
#Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in northYork_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = northYork_grouped[northYork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [20]:
#Let's put that into a pandas dataframe
#First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [21]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northYork_grouped['Neighborhood']

for ind in np.arange(northYork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northYork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Middle Eastern Restaurant,Deli / Bodega,Bridal Shop,Pharmacy,Restaurant,Sandwich Place,Shopping Mall,Pizza Place
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Butcher,Juice Bar,Café,Comfort Food Restaurant,Indian Restaurant,Pharmacy,Pizza Place
3,Don Mills,Japanese Restaurant,Coffee Shop,Asian Restaurant,Restaurant,Gym,Beer Store,Café,Dim Sum Restaurant,Discount Store,Concert Hall
4,Downsview,Grocery Store,Park,Baseball Field,Gym / Fitness Center,Discount Store,Electronics Store,Shopping Mall,Snack Place,Business Service,Bank


In [22]:
#Cluster Neighborhoods
#Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

northYork_grouped_clustering = northYork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northYork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 1, 0, 0, 0, 0, 1, 4, 3, 3])

In [31]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
# convert float to integer
kmeansInt = []
for val in kmeans.labels_:
    kmeansInt.append(int(val))
        

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeansInt)


#neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1)
northYork_merged = northYork_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northYork_merged = pd.merge(northYork_merged, neighborhoods_venues_sorted, on='Neighborhood', how='inner')

northYork_merged.head(200) # check the last columns!


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Park,Construction & Landscaping,Food & Drink Shop,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Women's Store,Dim Sum Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Accessories Store,Coffee Shop,Boutique,Furniture / Home Store,Event Space,Miscellaneous Shop,Vietnamese Restaurant,Bank,Dog Run
3,M3B,North York,Don Mills,43.745906,-79.352188,0,Japanese Restaurant,Coffee Shop,Asian Restaurant,Restaurant,Gym,Beer Store,Café,Dim Sum Restaurant,Discount Store,Concert Hall
4,M3C,North York,Don Mills,43.725900,-79.340923,0,Japanese Restaurant,Coffee Shop,Asian Restaurant,Restaurant,Gym,Beer Store,Café,Dim Sum Restaurant,Discount Store,Concert Hall
5,M6B,North York,Glencairn,43.709577,-79.445073,1,Japanese Restaurant,Asian Restaurant,Pub,Park,Women's Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
6,M2H,North York,Hillcrest Village,43.803762,-79.363452,4,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
7,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,0,Bank,Coffee Shop,Middle Eastern Restaurant,Deli / Bodega,Bridal Shop,Pharmacy,Restaurant,Sandwich Place,Shopping Mall,Pizza Place
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Women's Store,Bakery,Food Court,Cosmetics Shop,Japanese Restaurant,Jewelry Store
9,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,0,Coffee Shop,Furniture / Home Store,Miscellaneous Shop,Caribbean Restaurant,Metro Station,Massage Studio,Bar,Dim Sum Restaurant,Concert Hall,Construction & Landscaping


In [24]:
#Finally, let's visualize the resulting clusters
# create map
import math
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northYork_merged['Latitude'], northYork_merged['Longitude'], northYork_merged['Neighborhood'], northYork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
           
           
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters